In [1]:
from huggingface_hub import hf_hub_download
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from transformer_lens import HookedTransformer
model_name = "EleutherAI/pythia-70m-deduped"

model = HookedTransformer.from_pretrained(model_name, device=device)

Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


In [4]:
# Downnload dataset
from datasets import Dataset, load_dataset
dataset_name = "JeanKaddour/minipile"
token_amount= 40
#TODO: change train[:1000] to train if you want whole dataset
# 100_000 datasets
# I think that we want to use the full 100_000 at some point...
# dataset = load_dataset(dataset_name, split="train[:100000]").map(
dataset = load_dataset(dataset_name, split="train[:10000]").map( # 1_000 to get started
    lambda x: model.tokenizer(x['text']),
    batched=True,
).filter(
    lambda x: len(x['input_ids']) > token_amount
).map(
    lambda x: {'input_ids': x['input_ids'][:token_amount]}
)
# TODO: we can maybe make this faster for the larger dataset?

Found cached dataset parquet (/home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ad2ecfd158f710eb.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d173f762c78db5b8.arrow
Loading cached processed dataset at /home/lev/.cache/huggingface/datasets/JeanKaddour___parquet/JeanKaddour--minipile-0d7d2d1ff79d1d36/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-bd49b6b2e3856158.arrow


In [5]:
setting = "residual"

def get_cache_name_neurons(layer: int):
    if setting == "residual":
        cache_name = f"blocks.{layer}.hook_resid_post"
        neurons = model.cfg.d_model
    elif setting == "mlp":
        cache_name = f"blocks.{layer}.mlp.hook_post"
        neurons = model.cfg.d_mlp
    elif setting == "attention":
        cache_name = f"blocks.{layer}.hook_attn_out"
        neurons = model.cfg.d_model
    elif setting == "mlp_out":
        cache_name = f"blocks.{layer}.hook_mlp_out"
        neurons = model.cfg.d_model
    else:
        raise NotImplementedError
    return cache_name, neurons

In [6]:
n_layers = model.cfg.n_layers
model.cfg.d_model, n_layers

(512, 6)

# Get Dictionary Activations

In [7]:
# TODO: in chunks...
# TODO: cache?

# Now we can use the model to get the activations
from torch.utils.data import DataLoader
from datasets import DatasetDict
from tqdm.auto import tqdm
from einops import rearrange
import math

# MAX_CHUNK_SIZE = 1_000

# TODO: move to a separate file or something
def get_activations(layer: int):
    datapoints = dataset.num_rows
    embedding_size = model.cfg.d_model
    activations_final = np.memmap(f'layer-{layer}.mymemmap', dtype='float32', mode='w+', shape=(datapoints, token_amount, embedding_size))
    batch_size = 32

    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        cache_name = get_cache_name_neurons(layer)[0]
        for i, batch in enumerate(tqdm(dl)):
            # print(batch)
            _, cache = model.run_with_cache(batch.to(device))
            # print("AA", cache[cache_name].shape)
            # batched_neuron_activations = rearrange(cache[cache_name], "b s n -> (b s) n" )

            real_batch_size = batch.shape[0]
            activations_final[i*batch_size:i*batch_size + real_batch_size, :, :] = cache[cache_name].cpu().numpy()
    return activations_final

model_activations = [get_activations(layer) for layer in range(n_layers)]

  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 0/310 [00:00<?, ?it/s]

  0%|          | 0/310 [00:00<?, ?it/s]

## Params

In [8]:
LAYER = 0

## Get activations for a specific feature and visualize them

In [9]:
model_activations[0].shape, model_activations[LAYER].reshape(-1, model_activations[LAYER].shape[-1]).shape

((9909, 40, 512), (396360, 512))

In [10]:
from interp_utils import *
import torch
import numpy as np

# Get the activations for the best dict features
def get_feature_datapoints_with_idx(feature_index, dictionary_activations, tokenizer, token_amount, dataset, k=10, setting="max"):
    if len(dictionary_activations.shape) == 3:
        best_feature_activations = dictionary_activations[:, :, feature_index].flatten()
    else:
        best_feature_activations = dictionary_activations
    # Sort the features by activation, get the indices
    if setting=="max":
        # TODO:! Urrr.... is this backwards? CHECK IF ::-1 is correct but I think that it is
        found_indices = np.argsort(best_feature_activations)[::-1][:k]
        # found_indices = np.argsort(best_feature_activations, descending=True)[:k]
    elif setting=="uniform":
        # min_value = torch.min(best_feature_activations)
        min_value = np.min(best_feature_activations)
        max_value = np.max(best_feature_activations)

        # Define the number of bins
        num_bins = k

        # Calculate the bin boundaries as linear interpolation between min and max
        bin_boundaries = np.linspace(min_value, max_value, num_bins + 1)

        # Assign each activation to its respective bin
        # TODO: hmm
        # np bucketize?
        # bins = torch.bucketize(best_feature_activations, bin_boundaries)
        bins = np.digitize(best_feature_activations, bin_boundaries)

        # Initialize a list to store the sampled indices
        sampled_indices = []

        # Sample from each bin
        for bin_idx in np.unique(bins):
            if(bin_idx==0): # Skip the first one. This is below the median
                continue
            # Get the indices corresponding to the current bin
            bin_indices = np.array(np.nonzero(bins == bin_idx)).squeeze(axis=0)
            # print(bin_indices.shape)
            
            # Randomly sample from the current bin
            sampled_indices.extend(np.random.choice(bin_indices, size=1, replace=False))

        # Convert the sampled indices to a PyTorch tensor & reverse order
        found_indices = np.flip(np.array(sampled_indices), axis=[0])
    else: # random
        # get nonzero indices
        nonzero_indices = torch.nonzero(best_feature_activations)[:, 0]
        # shuffle
        shuffled_indices = nonzero_indices[torch.randperm(nonzero_indices.shape[0])]
        found_indices = shuffled_indices[:k]
    num_datapoints = int(dictionary_activations.shape[0])
    datapoint_indices =[np.unravel_index(i, (num_datapoints, token_amount)) for i in found_indices]
    text_list = []
    full_text = []
    token_list = []
    full_token_list = []
    for md, s_ind in datapoint_indices:
        md = int(md)
        s_ind = int(s_ind)
        full_tok = torch.tensor(dataset[md]["input_ids"])
        full_text.append(tokenizer.decode(full_tok))
        tok = dataset[md]["input_ids"][:s_ind+1]
        text = tokenizer.decode(tok)
        text_list.append(text)
        token_list.append(tok)
        full_token_list.append(full_tok)
    return text_list, full_text, token_list, full_token_list, found_indices

## Baseline before looking at "deconstructive interference"

In [11]:
import interp_utils
import importlib
importlib.reload(interp_utils)

feature = 69
text_list, full_text, token_list, full_token_list, indices = get_feature_datapoints_with_idx(feature, model_activations[LAYER], model.tokenizer, token_amount, dataset, setting="uniform", k=100)
interp_utils.visualize_text(text_list, feature, model, None, layer=LAYER, setting="model")

## Idea
We can maybe do a basic K-means on just **layers**'s activations. We use this to get a sense for "neuron's which fire together, wire together".
Maybe we do not have to use K-Means but can instead build some sort of relationship graph.

### Then:
- We find the top sentences which activate the clustered neurons!

## Redifine Params for the $``\text{Idea}"$

In [34]:
NEURON = 69
LAYER = 0
N_RELATED = 10
N_CLUSTER_EXAMPLES = 5_000

## Setup correlations and such

In [44]:
## Compute the correlations
import numpy as np
from scipy.stats import pearsonr

# Assuming 'activations' is your numpy array of shape (num_samples, num_neurons)
# where each row is an activation vector for all neurons

def compute_correlations(activations, neuron=NEURON):
    num_neurons = activations.shape[1]
    neur_acts = activations[:, neuron]
    corr_list = np.zeros(num_neurons)
    for i in range(num_neurons):
      corr, _ = pearsonr(activations[:, i], neur_acts)
      corr_list[i] = corr
    return corr_list

neuron_corrs = compute_correlations(model_activations[LAYER].reshape(-1, model_activations[LAYER].shape[-1]))
top_correlated_neurons = np.argsort(np.abs(neuron_corrs))[::-1][:N_RELATED]
neuron_corrs[top_correlated_neurons]

array([1.])

## Looking at constructive interference

In [36]:
def get_top_related_datapoints(correlated_indices: list[int], layer=LAYER, n_examples=100):
	# TODO: np.abs(*)?
	feature_activations = np.abs(model_activations[layer][:, :, correlated_indices])
	summed_along_sentence = feature_activations.sum(axis=1)
	weighted = summed_along_sentence * neuron_corrs[correlated_indices]
	summed_along_features = weighted.sum(axis=1)
	# Find the input data-points that most activate the neuron
	found_indices = np.argsort(summed_along_features)[::-1][:n_examples]
	return found_indices


def get_relevant_neurons(correlated_indices: list[int], layer=LAYER, n_examples=100, weight_cutoff=1.2): # TODO: check weight cutoff vis a vis using quantified models
	# Get the input data-points that most activate the neuron
	found_indices = get_top_related_datapoints(correlated_indices, layer=layer, n_examples=n_examples)

	def get_activated_neurons(layer: int):
		neurons = set()
		for i in found_indices:
			cutoff_n = model_activations[layer][i, :, :] > weight_cutoff
			_pos_nonzero, neuron_nonzero = np.nonzero(cutoff_n)
			# print("LEN", neuron_nonzero.shape)
			neurons.update(neuron_nonzero)
		return list(neurons)
	
	other_layer_neurons = []
	for i in range(n_layers):
		# if i != layer:
		other_layer_neurons.append((i, get_activated_neurons(i)))
	return other_layer_neurons

other_neurons = get_relevant_neurons(correlated_indices=top_correlated_neurons, layer=LAYER, n_examples=N_CLUSTER_EXAMPLES, weight_cutoff=2)
len(other_neurons[4][1]), [len(i[1]) for i in other_neurons]

(462, [10, 84, 274, 314, 462, 508])

In [37]:
go_to_n_layers = 4

def get_auxiliary_data(correlated_indices: list[int], layer: int, n_examples=N_CLUSTER_EXAMPLES):
	# TODO: this should be a different function!! (UTILS)
	# Get the input data-points that most activate the neuron
	# TODO: abs
	# best_feature_activations = np.abs(model_activations[layer][:, :, correlated_indices])
	# # TODO: weight by correlation?!
	# summed_along_sentence_and_correlates = best_feature_activations.sum(axis=1).sum(axis=1)
	# # Find the input data-points that most activate the neuron
	# found_indices = np.argsort(summed_along_sentence_and_correlates)[::-1][:n_examples]

	# Get the input data-points that most activate the neuron
	found_indices = get_top_related_datapoints(correlated_indices, layer=layer, n_examples=n_examples)

	layer_idx = {i: layer for i, layer in enumerate(
		list({layer for i, (layer, _) in enumerate(other_neurons)})
	)}

	print(layer_idx, layer_idx.keys())
	layer_dividers_list = []
	on_layer = -1

	total_other_neurons = 0
	for other_neur in other_neurons:
		other_layer, neurons = other_neur
		if other_layer <= go_to_n_layers:
			if other_layer != on_layer:
				on_layer = other_layer
				layer_dividers_list.append(total_other_neurons)
			total_other_neurons += len(neurons)
	layer_dividers_list.append(total_other_neurons)

	layer_dividers = {}
	print(layer_dividers_list)
	# TODO: with go_to_n_layers there is going to be some overflow problems
	for i in range(go_to_n_layers):
		layer_dividers[layer_idx[i]] = (layer_dividers_list[i],  layer_dividers_list[i+1])
			
	# sum([len(i[1]) for i in other_neurons])


	concatenated = np.zeros((len(found_indices), total_other_neurons))

	counter = 0
	for i, other_neur in enumerate(other_neurons):
		other_layer, neurons = other_neur
		if other_layer <= go_to_n_layers:
			r = model_activations[other_layer][:, :, neurons][found_indices].sum(axis=1) # Sum over the entire sentence/ text input
			concatenated[:, counter:counter+len(neurons)] = r
			counter += len(neurons)
		
	return concatenated, found_indices, layer_dividers


aux_data, datapoints_used, layer_dividers  = get_auxiliary_data(correlated_indices=top_correlated_neurons, layer=0)
# TODO: CONSIDER ONLY USING THE CLOSER LAYERS...
print(aux_data.shape), print(datapoints_used.shape), print(layer_dividers)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5} dict_keys([0, 1, 2, 3, 4, 5])
[0, 10, 94, 368, 682, 1144]
(5000, 1144)
(5000,)
{0: (0, 10), 1: (10, 94), 2: (94, 368), 3: (368, 682)}


(None, None, None)

In [38]:
def run_gaussian_mixture_model():
	# TODO:
	pass

In [39]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

# Cosine similarity function
# TODO: yay or no

def create_per_layer_weighting():
    weighting = np.ones(aux_data.shape[-1])
    for curr_layer in layer_dividers:
        start, stop = layer_dividers[curr_layer]
        # The further from the current layer, the less important
        weighting[start:stop] = weighting[start:stop] * (2 ** (-1 * (abs(LAYER - curr_layer))))
        # TODO: not just per neuron
    return weighting

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def weighted_dot_similarity(weighting, a, b):
    ab = a * b 
    weighted = ab * weighting
    return np.sum(weighted)
    # return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# KMeans with cosine similarity


def kmeans_cosine(X, n_clusters: int, iterations=100):
    k = n_clusters
    # Normalize input data
    # TODO: why are we normalizing??
    X_normalized = normalize(X, axis=1)
    weighting = create_per_layer_weighting()

    # Randomly initialize centroids
    n_samples, n_features = X_normalized.shape
    centroids = X_normalized[np.random.choice(n_samples, k, replace=False)]

    for iter in range(iterations):
        # Cluster assignment step
        clusters = [[] for _ in range(k)]
        for idx, x in enumerate(X_normalized):
            similarities = [weighted_dot_similarity(weighting,
                x, centroid) for centroid in centroids]
            # similarities = [np.dot(x, centroid) for centroid in centroids]
            closest = np.argmax(similarities)
            clusters[closest].append(idx)

        # Update centroids
        # TODO: we maybe able to just **not use** PCA at all here.... slow it may be
        new_centroids = []
        for cluster in clusters:
            if cluster:  # Check if cluster is not empty
                new_centroid = np.mean(X_normalized[cluster], axis=0)
                new_centroids.append(new_centroid)
            else:
                # Reinitialize empty clusters
                new_centroids.append(np.random.rand(n_features))

        new_centroids = np.array(new_centroids)
        if np.allclose(centroids, new_centroids):
            break
        centroids = new_centroids
        print("Done with iteration", iter)

    return centroids, clusters


# TODO: no function. Just on global so we can stop middway etc etc
# TODO: can we speed this up??? Maybe we use PCA
_, cluster_by_idx = kmeans_cosine(aux_data, iterations=400, n_clusters=30)

Done with iteration 0
Done with iteration 1
Done with iteration 2
Done with iteration 3
Done with iteration 4
Done with iteration 5
Done with iteration 6
Done with iteration 7
Done with iteration 8
Done with iteration 9
Done with iteration 10
Done with iteration 11
Done with iteration 12
Done with iteration 13
Done with iteration 14
Done with iteration 15
Done with iteration 16
Done with iteration 17
Done with iteration 18
Done with iteration 19
Done with iteration 20
Done with iteration 21
Done with iteration 22
Done with iteration 23
Done with iteration 24
Done with iteration 25
Done with iteration 26
Done with iteration 27
Done with iteration 28
Done with iteration 29
Done with iteration 30
Done with iteration 31
Done with iteration 32


In [40]:
# TODO: GMMs
print([len(c) for c in cluster_by_idx])
for i, c in enumerate(cluster_by_idx):
	if len(c) > 0:
		print(i)

[12, 4, 244, 378, 4, 20, 1843, 28, 23, 32, 36, 1, 11, 33, 41, 699, 85, 87, 37, 15, 5, 103, 300, 812, 9, 22, 33, 67, 12, 4]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [43]:
# TODO: we have to change how ``best'' is done and only visualize within our clusters, not
# just sample from the cluster and visualize everything

# also, it seems like abs(*) is not really helping??? Idk,

cluster_idx = 0

cluster_inds = [int(datapoints_used[int(cluster_by_idx[cluster_idx][i])]) for i in range(len(cluster_by_idx[cluster_idx]))]
print(dataset[int(cluster_inds[0])]['text'][:100])

new_dataset = []
for i in cluster_inds:
	new_dataset.append(dataset[i])
# for i in range(len(cluster_by_idx[cluster_idx])):
# 	print(dataset[int(datapoints_used[int(cluster_by_idx[cluster_idx][i])])]['text'][:100])
# 	print("+++++++++++++++++++++++++++++++++++++++++++++++++")

activations = model_activations[LAYER][cluster_inds, :, :]
# .reshape(-1, model_activations[layer].shape[-1])[cluster, :], 1)
print(activations.shape)

# TODO: weighted correlations here
text_list, full_text, token_list, full_token_list, indices = get_feature_datapoints_with_idx(NEURON, activations, model.tokenizer, token_amount, new_dataset, setting="uniform", k=30)
interp_utils.visualize_text(text_list, feature, model, None, layer=LAYER, setting="model")
# TODO: maybe do everything on MLP side where we get only positive activations

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd
(12, 40, 512)
